##INSTALL

##IMPORTS

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from xgboost import XGBClassifier
import joblib
import numpy as np

##LOAD DATA

In [20]:
df = pd.read_csv("/content/creditcard.csv")

### Select valid features

In [21]:
features = ["Time"] + [f"V{i}" for i in range(1, 29)] + ["Amount"]
X = df[features]
y = df["Class"]

##Split data

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

### Scale numeric features

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, "scaler.pkl")

print("Scaler saved as scaler.pkl")

Scaler saved as scaler.pkl


##Logistic Regression

In [24]:
logreg = LogisticRegression(
    class_weight="balanced", max_iter=2000, random_state=42
)
logreg.fit(X_train_scaled, y_train)
joblib.dump(logreg, "logistic_regression.pkl")

print("Saved logistic_regression.pkl")

Saved logistic_regression.pkl


##Random Forest

In [25]:
rf = RandomForestClassifier(
    n_estimators=300, class_weight="balanced", random_state=42
)
rf.fit(X_train, y_train)
joblib.dump(rf, "random_forest.pkl")

print("Saved random_forest.pkl")

Saved random_forest.pkl


##XGBoost

In [26]:
xgb = XGBClassifier(
    n_estimators=300,
    scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train),
    random_state=42,
    eval_metric="logloss",
)
xgb.fit(X_train, y_train)
joblib.dump(xgb, "xgboost.pkl")

print("Saved xgboost.pkl")

Saved xgboost.pkl


##Isolation Forest

In [27]:
iso = IsolationForest(
    n_estimators=200, contamination=0.001, random_state=42
)
iso.fit(X_train)
joblib.dump(iso, "isolation_forest.pkl")

print("Saved isolation_forest.pkl")

Saved isolation_forest.pkl


##Hybrid Model

In [28]:
class HybridModel:
    def __init__(self, rf_model, xgb_model, iso_model):
        self.rf = rf_model
        self.xgb = xgb_model
        self.iso = iso_model

    def predict(self, X):
        p1 = self.rf.predict_proba(X)[:, 1]
        p2 = self.xgb.predict_proba(X)[:, 1]
        p3 = np.where(self.iso.predict(X) == -1, 1, 0)

        avg = (p1 + p2 + p3) / 3
        return (avg > 0.5).astype(int)

    def predict_proba(self, X):
        p1 = self.rf.predict_proba(X)[:, 1]
        p2 = self.xgb.predict_proba(X)[:, 1]
        p3 = np.where(self.iso.predict(X) == -1, 1, 0)

        avg = (p1 + p2 + p3) / 3
        return np.vstack([1 - avg, avg]).T


hybrid = HybridModel(rf, xgb, iso)
joblib.dump(hybrid, "hybrid_model.pkl")

print("Saved hybrid_model.pkl")

Saved hybrid_model.pkl
